<figure>
<img src="../Imagenes/logo-final-ap.png"  width="80" height="80" align="left"/> 
</figure>

# <span style="color:blue"><left>Aprendizaje Profundo</left></span>

# <span style="color:red"><center>Introducción a Redes Neuronales Recurrentes</center></span>

##   <span style="color:blue">Profesores</span>

1. Alvaro Mauricio Montenegro Díaz, ammontenegrod@unal.edu.co
2. Daniel Mauricio Montenegro Reyes, dextronomo@gmail.com 
3. Campo Elías Pardo Turriago, cepardot@unal.edu.co 

##   <span style="color:blue">Asesora Medios y Marketing digital</span>
 

4. Maria del Pilar Montenegro, pmontenegro88@gmail.com 

## <span style="color:blue">Asistentes</span>

5. Oleg Jarma, ojarmam@unal.edu.co 
6. Laura Lizarazo, ljlizarazore@unal.edu.co 

## <span style="color:blue">Referencias</span>

1. [Introducción a Redes LSTM](Intro_LSTM.ipynb)
2. [Time Series Forecasting with LSTMs using TensorFlow 2 and Keras in Python](https://towardsdatascience.com/time-series-forecasting-with-lstms-using-tensorflow-2-and-keras-in-python-6ceee9c6c651/)
3. [Dive into Deep Learnig](https://d2l.ai/)

## <span style="color:blue">Contenido</span>

* [Introducción](#Introducción)
* [Redes Neuronales con una capa oculta](#Redes-Neuronales-con-una-capa-oculta)
* [Redes Neuronales con estados ocultoss](#Redes-Neuronales-con-estados-ocultos)
* [Redes Neuronales basados en Modelos de lenguaje caracter-etiqueta](#Redes-Neuronales-basados-en-Modelos-de-lenguaje-caracter-etiqueta)
* [Perplexity](#Perplexity)
* [Resumen](#Resumen)
* [Ejercicios](#Ejercicios)

## <span style="color:blue">Introducción</span>

En la introducción a procesamiento de lenguaje natural introducimos los modelos $ n $-gram. En tales modelos la probabilidad condicional de la palabra $ x_t $ en el paso de tiempo $ t $ solo depende de $ n-1 $ las palabras anteriores .


Si queremos incorporar el posible efecto de palabras anteriores al paso de tiempo $ t-(n-1) $ en $ x_t $,
necesitamos aumentar $ n $.

Sin embargo, el número de parámetros del modelo también aumentaría exponencialmente con él, ya que necesitamos almacenar $ | \mathcal{V} |^n $ números para un conjunto de vocabulario $ \mathcal{V} $.
Por lo tanto, en lugar de modelar $ P (x_t \mid x_ {t-1}, \ldots, x_{t-n + 1}) $, es preferible usar un modelo de variable latente:

$$ P (x_t \mid x_ {t-1}, \ldots, x_1) \approx P (x_t \mid h_ {t-1}), $$

en donde $ h_{t-1} $ es un `estado oculto` (también conocido como variable oculta) que almacena la información de la secuencia hasta el paso de tiempo $ t-1 $.

En general, el estado oculto en cualquier paso de tiempo $ t $ podría calcularse basándose tanto en la entrada actual $ x_{t} $ como en el estado oculto anterior $ h_{t-1} $:

$$ h_t = f (x_{t}, h_{t-1}). $$


Para una función suficientemente poderosa $ f $, el modelo de variable latente no es una aproximación. Después de todo, $ h_t $ simplemente puede almacenar todos los datos que ha observado hasta ahora.
Sin embargo, potencialmente podría encarecer tanto la computación como el almacenamiento.

Recuerde que hemos hablado de capas ocultas con unidades ocultas cuando introducimos los  perceptrones multicapa.

Cabe destacar que capas ocultas y estados ocultos se refieren a dos conceptos muy diferentes.

+ Las capas ocultas son, como se explicó, capas que están ocultas a la vista en la ruta desde la entrada hasta la salida.
+ Los estados ocultos son técnicamente hablando `entradas` a todo lo que hacemos en un paso dado, y solo se pueden calcular observando los datos de los pasos de tiempo anteriores.

Las `redes neuronales recurrentes` (RNR) son redes neuronales con estados ocultos. Antes de presentar el modelo RNN, revisitemos el modelo perceptron multicapa (PMC).

## <span style="color:blue">Redes Neuronales con una capa oculta</span> 

Damos un vistazo a un MLP con una sola capa oculta. Supongamos  que la función de activación de la capa oculta es $ \phi $.

Dado un mini lote de ejemplos $\mathbf{X} \in \mathbb{R}^{n \times d}$ con tamaño de lote $n$ y $d$ entradas, la salida de la capa oculta, denotada por  $\mathbf{H} \in \mathbb{R}^{n \times h}$ se calcula como

$$\mathbf{H} = \phi(\mathbf{X} \mathbf{W}_{xh} + \mathbf{b}_h).$$


En esta ecuación tenemos que el parámetro de pesos es $\mathbf{W}_{xh} \in \mathbb{R}^{d \times h}$, el sesgo (bias) es  $\mathbf{b}_h \in \mathbb{R}^{1 \times h}$, y el número de unidades ocultas $h$, para la capa oculta.


Tenga en cuenta que, difusión (broadcasting) es aplicado durante la suma.

Ahora, la variable oculta $\mathbf{H}$ se utiliza como entrada de la capa de salida. La capa de salida está dada por

$$\mathbf{O} = \mathbf{H} \mathbf{W}_{hq} + \mathbf{b}_q,$$

en donde $\mathbf{O} \in \mathbb{R}^{n \times q}$ es la variable de salida, $\mathbf{W}_{hq} \in \mathbb{R}^{h \times q}$ es el parámetro de peso, y $\mathbf{b}_q \in \mathbb{R}^{1 \times q}$ es el parámetro de sesgo de la capa de salida.  Si se trata de un problema de clasificación, podemos utilizar $\text{softmax}(\mathbf{O})$ para calcular la distribución de probabilidad de las categorías de salida.

Esto es completamente análogo al problema de regresión que resolvimos anteriormente en la introducción a secuencias, por lo que omitimos detalles.
Baste decir que podemos elegir pares de características y etiquetas al azar y aprender los parámetros de nuestra red a través de la diferenciación automática y el descenso de gradiente estocástico.

## <span style="color:blue">Redes Neuronales con estados ocultos</span> 

Las cosas son completamente diferentes cuando tenemos estados ocultos. Veamos la estructura con más detalle.

Supongamos que tenemos un minilote de entradas $ \mathbf {X}_t \in \mathbb {R} ^ {n \times d} $
en el paso de tiempo $ t $. En otras palabras, para un minilote de $ n $ ejemplos de secuencia, 
cada fila de $ \mathbf {X}_t $ corresponde a un ejemplo en el paso de tiempo $ t $ de la secuencia.

En adelante  $ \mathbf {H} _t \in \mathbb {R} ^ {n \times h} $ denotará la variable oculta en el  paso de tiempo $ t $.

A diferencia del MLP, aquí guardamos la variable oculta $ \mathbf {H}_{t-1} $ del paso de tiempo anterior e introducimos un nuevo parámetro de peso $ \mathbf {W}_{hh} \in \mathbb {R }^{h \times h}$ para describir cómo usar la variable oculta del paso de tiempo anterior en el paso de tiempo actual. 

Específicamente, el cálculo de la variable oculta del paso de tiempo actual está determinado por la entrada del paso de tiempo actual junto con la variable oculta del paso de tiempo anterior:

$$ \mathbf {H}_t = \phi (\mathbf {X} _t \mathbf {W}_{xh} + \mathbf {H}_{t-1} \mathbf {W}_{hh} + \mathbf {b}_h). $$


En comparación con la ecuación de la salida de la capa oculta arriba, esta ecuación   agrega el término $ \mathbf{H}_{t-1} \mathbf {W}_{hh} $.


De la relación entre las variables ocultas $ \mathbf{H}_t $ y $ \mathbf{H}_{t-1} $ de pasos de tiempo adyacentes, sabemos que estas variables capturaron y retuvieron la información histórica de la secuencia hasta su paso de tiempo actual, al igual que el estado o la memoria del paso de tiempo actual de la red neuronal. 

Por lo tanto, esta variable oculta se denomina `estado oculto`. Dado que el estado oculto usa la misma definición del paso de tiempo anterior en el paso de tiempo actual, el cálculo de $ \mathbf {H}_t$ es recurrente.  Por tanto, las redes neuronales con estados ocultos basados en cálculos recurrentes se nombran `redes neuronales recurrentes`.


Capas que ejecutan el cálculo de esta última ecuación se llaman *capas recurrentes*.


Hay muchas formas diferentes de construir RNR. Las RNR con un estado oculto definido arriba  son muy comunes.


Para el paso de tiempo $ t $, la salida de la capa de salida es similar al cálculo en el PMC:

$$\mathbf{O} _t = \mathbf {H}_t \mathbf {W}_{hq} + \mathbf {b}_q. $$

Los parámetros de la RNR incluyen los pesos $ \mathbf{W}_{xh} \in \mathbb{R}^{d \times h}, \mathbf{W}_{hh} \in \mathbb{R}^{h \times q}$ y el sesgo $ \mathbf{b}_h \in \mathbb{R}^{1 \times h} $
de la capa oculta, junto con los pesos $ \mathbf {W} _ {hq} \in \mathbb {R} ^ {h \times q} $
y el sesgo $ \mathbf{b}_q \in \mathbb{R}^{1 \times q} $ de la capa de salida.


Vale la pena mencionar que incluso en diferentes pasos de tiempo, Los RNR siempre usan estos parámetros de modelo. Por tanto, el costo de parametrización de un RNR no crece a medida que aumenta el número de pasos de tiempo.

La siguiente figura ilustra la lógica computacional de un RNR en tres pasos de tiempo adyacentes.
En cualquier momento paso $ t $, el cálculo del estado oculto se puede entender como:

1. concatenando la entrada $ \mathbf {X}_t $ en el paso de tiempo actual $ t $ y el estado oculto $ \mathbf {H}_{t-1} $ en el paso de tiempo anterior $ t-1 $;
1.  alimentar el resultado de la concatenación en una capa completamente conectada con la función de activación $ \phi $.


<figure>
<center>
<img src="../Imagenes/rnn.svg" width="500" height="500" align="center"/>
</center>
</figure>

**<center >Red Neuronal recurrente con un estado oculto.</center>**
Fuente: [Dive into Deep learning](https://d2l.ai/)


La salida de tal capa completamente conectada es el estado oculto $ \mathbf {H}_t $ del paso de tiempo actual $ t $. En este caso, los parámetros del modelo son la concatenación de $ \mathbf{W}_{xh} $ y $\mathbf{W}_{hh} $, y un sesgo de $ \mathbf{b}_h $, todo dela última ecuación.

El estado oculto del paso de tiempo actual $ t $, $ \mathbf{H}_t $, participará en el cálculo del estado oculto $ \mathbf {H}_{t + 1} $ del próximo paso de tiempo $ t + 1$. Además, $ \mathbf{H}_t $ también será alimentado en la capa de salida totalmente conectada para calcular la salida
$ \mathbf{O}_t $ del paso de tiempo actual $ t $.




Acabamos de mencionar que el cálculo de 
$\mathbf{X}_t \mathbf{W}_{xh} + \mathbf{H}_{t-1} \mathbf{W}_{hh}$ 
para el estado oculto es equivalente a multiplicación de matriz de  concatenación de $ \mathbf{X}_t $ y $ \mathbf{H}_{t-1} $ y concatenación de $ \mathbf{W}_{xh} $ y $ \mathbf{W}_{hh}$.

Aunque esto se puede demostrar en matemáticas, a continuación, solo usamos un fragmento de código simple para mostrar esto. Para empezar, definimos las matrices $X$, $W_{xh}$, $H$ y $W_{hh}$, cuyas formas son (3, 1), (1, 4), (3, 4) y (4, 4), respectivamente.

Multiplicar `X` por `W_xh` y `H` por `W_hh`, respectivamente, y luego sumar estas dos multiplicaciones,`X`,` W_xh` obtenemos una matriz de forma (3, 4).

In [2]:
import tensorflow as tf

X, W_xh = tf.random.normal((3, 1), 0, 1), tf.random.normal((1, 4), 0, 1)
H, W_hh = tf.random.normal((3, 4), 0, 1), tf.random.normal((4, 4), 0, 1)
tf.matmul(X, W_xh) + tf.matmul(H, W_hh)

<tf.Tensor: shape=(3, 4), dtype=float32, numpy=
array([[-0.9376621 , -0.37132972,  0.6113366 , -1.1120977 ],
       [ 1.5709383 ,  0.07383901, -0.30149186, -1.9105288 ],
       [ 0.49457762,  0.20922334, -0.6078828 ,  0.9659305 ]],
      dtype=float32)>

Ahora concatenamos las matrices `X` y` H`
a lo largo de columnas (eje 1),
y las matrices
`W_xh` y` W_hh` a lo largo de las filas (eje 0).
Estas dos concatenaciones
resulta en
matrices de forma (3, 5)
y de forma (5, 4), respectivamente.
Multiplicando estas dos matrices concatenadas,
obtenemos la misma matriz de salida de forma (3, 4)
como anteriormente.


In [3]:
tf.matmul(tf.concat((X, H), 1), tf.concat((W_xh, W_hh), 0))

<tf.Tensor: shape=(3, 4), dtype=float32, numpy=
array([[-0.9376621 , -0.37132967,  0.6113366 , -1.1120977 ],
       [ 1.5709385 ,  0.07383901, -0.30149186, -1.9105288 ],
       [ 0.49457765,  0.20922334, -0.60788274,  0.96593046]],
      dtype=float32)>

## <span style="color:blue">Redes Neuronales basados en Modelos de lenguaje caracter-etiqueta</span>

Recuerde que para el modelado de idiomas nuestro objetivo es predecir el siguiente token basado en
los tokens actuales y anteriores, Así cambiamos la secuencia original por un token.
como las etiquetas. [Bengio et al](https://www.jmlr.org/papers/volume3/bengio03a/bengio03a.pdf). fueron los primeros en proponer usar una red neuronal para el modelado de idiomas.

A continuación ilustramos cómo se pueden usar RNN para predecier el siguiente caracter en un modelo de idioma basado en caracteres. Dejaremos que el tamaño del minilote sea uno, y la secuencia del texto sea "machine".

Para simplificar, tokenizamos el texto en caracteres en lugar de las palabras y consideramos un `modelo de lenguaje de nivel de carácter`.
La siguiente figura muestra cómo predecir el siguiente carácter en función de los caracteres actuales y anteriores a través de una RNR para el modelado de lenguaje de nivel de carácter.


<figure>
<center>
<img src="../Imagenes/rnn-train.svg" width="500" height="500" align="center"/>
</center>
</figure>

**<center >RNR para predicción del siguiente caracter con un modelo de lenguaje a nivel de caracter.</center>**
Fuente: [Dive into Deep learning](https://d2l.ai/)


Durante el proceso de entrenamiento,
Ejecuamos una operación `SoftMax` en la salida de la capa de salida para cada paso de tiempo, y luego use la pérdida de entropía cruzada para calcular el error entre la salida del modelo y la etiqueta.

Debido al cálculo recurrente del estado oculto en la capa oculta, la salida del tiempo en el paso 3 en la figura, es $ \mathbf{O} _3$, y está determinado por la secuencia de texto "M", "A", y "C". Dado que el siguiente carácter de la secuencia en los datos de capacitación es "H", la pérdida de tiempo, el paso 3 dependerá de la distribución de probabilidad del siguiente carácter generado en función de la secuencia de características "M", "A", "C" y La etiqueta "H" de este paso de tiempo.

En la práctica, cada token está representado por un vector $ d $-dimensional, y usamos un tamaño de lote $ n> 1 $. Por lo tanto, la entrada $ \mathbf{X}_t $ en el momento $ t $ será una matriz de $ n \times d $, que es idéntica a la que discutimos antes.

## <span style="color:blue">Perplexity</span>


Por último, analicemos cómo medir la calidad del modelo de lenguaje, que se utilizará para evaluar nuestros modelos basados en RNR en las secciones siguientes.

Una forma de evaluar la calidad del modelo de lenguaje es calcular lo sorprendente que resulta el texto. Un buen modelo de lenguaje puede predecir con tokens de alta precisión como revisamos a continuación. Considere las siguientes continuaciones de la frase "Está lloviendo", según lo propuesto por diferentes modelos de lenguaje:

1. "Está lloviendo afuera"
1. "Está lloviendo banano"
1. "Está lloviendo piouw; kcj pwepoiut"

En términos de calidad, el ejemplo 1 es claramente el mejor. Las palabras son sensatas y lógicamente coherentes. Si bien es posible que no refleje con precisión qué palabra sigue semánticamente ("en San Francisco" y "en invierno" habrían sido extensiones perfectamente razonables), el modelo es capaz de capturar qué tipo de palabra sigue.

El ejemplo 2 es considerablemente peor al producir una extensión sin sentido. No obstante, al menos el modelo ha aprendido a deletrear palabras y cierto grado de correlación entre las palabras. Por último, el ejemplo 3 indica un modelo mal entrenado que no se ajusta a los datos correctamente.

Podríamos medir la calidad del modelo calculando la probabilidad de la secuencia.
Desafortunadamente, este es un número difícil de entender y de comparar.
Después de todo, es mucho más probable que ocurran secuencias más cortas que las más largas, de ahí la evaluación del modelo de la obra magna de Tolstoi
*La guerra y paz* producirá inevitablemente una probabilidad mucho menor que, por ejemplo, en la novela de Saint-Exupery *El Principito*. Lo que falta es el equivalente a un promedio.

Como sabemos de antes, La  *entropía mide el grado de sorpresa* de un resultado en una distribución discreta y *entropía cruzada mide el grado de sorpesa de un resultado de una distribución en relación con una distribución de referencia*.


Si queremos comprimir texto, podemos preguntar sobre el siguiente token dado el conjunto actual de tokens. Un mejor modelo de lenguaje debería permitirnos predecir el próximo token con mayor precisión. Por lo tanto, debería permitirnos gastar menos bits en comprimir la secuencia.


Entonces podemos medirlo por la pérdida de la entropía cruzada promediada sobre todos los $ n $ tokens de una secuencia:

$$\frac{1}{n} \sum_{t=1}^n -\log P(x_t \mid x_{t-1}, \ldots, x_1),$$

donde $ P $ viene dado por un modelo de lenguaje y $ x_t $ es el token real observado en el paso de tiempo $ t $ de la secuencia. Observe que para este cálculo se usa la entropía cruzada. La distribución de referencia es la codificación one-hot de la etiqueta correspondiente, en cada salida. Además $P(x_t \mid x_{t-1}, \ldots, x_1)$ se calcula a partir de $\text{Softmax}(h_t)$.

Esto hace que el rendimiento en documentos de diferentes longitudes sea comparable. Por razones históricas, los científicos en el procesamiento del lenguaje natural prefieren usar una cantidad llamada *perplexity*. En pocas palabras, es el exponencial de la anterior ecuacion:

$$\exp\left(-\frac{1}{n} \sum_{t=1}^n \log P(x_t \mid x_{t-1}, \ldots, x_1)\right).$$

La perplejidad (perplexity) puede entenderse mejor como la media armónica del número de opciones reales que tenemos al decidir qué token elegir a continuación. 

Veamos algunos casos:

* En el mejor de los casos, el modelo siempre estima perfectamente la probabilidad del token de etiqueta como 1. En este caso, la perplejidad del modelo es 1.
* En el peor de los casos, el modelo siempre predice la probabilidad del token de etiqueta como 0. En esta situación, la perplejidad es infinito positivo.
* En la línea de base, el modelo predice una distribución uniforme sobre todos los tokens disponibles del vocabulario. En este caso, la perplejidad (perplexity) es igual al número de tokens únicos del vocabulario. De hecho, si tuviéramos que almacenar la secuencia sin ninguna compresión, esto sería lo mejor que podríamos hacer para codificarla. Por lo tanto, esto proporciona un límite superior no trivial que cualquier modelo útil debe superar.


## <span style="color:blue">Resumen</span>

* Una red neuronal que utiliza computación recurrente para estados ocultos se denomina red neuronal recurrente (RNN).
* El estado oculto de un RNN puede capturar información histórica de la secuencia hasta el paso de tiempo actual.
* El número de parámetros del modelo RNN no aumenta a medida que aumenta el número de pasos de tiempo.
* Podemos crear modelos de lenguaje a nivel de personaje usando un RNN.
* Podemos utilizar la perplejidad para evaluar la calidad de los modelos lingüísticos.

## <span style="color:blue">Ejercicios</span>

1. Si usamos un RNN para predecir el siguiente carácter en una secuencia de texto, ¿cuál es la dimensión requerida para cualquier resultado?
1. ¿Por qué los RNN pueden expresar la probabilidad condicional de un token en algún paso de tiempo basándose en todos los tokens anteriores en la secuencia de texto?
1. ¿Qué le sucede al gradiente si se propaga hacia atrás a través de una secuencia larga?
1. ¿Cuáles son algunos de los problemas asociados con el modelo de lenguaje descrito en esta sección?